In [78]:
import pandas as pd
import numpy as np
import unicodedata
import re
import json

In [79]:
# Variáveis Globais
ABA_CAPITAIS_FULL                   = "MEXER_NESSA"
DATA_PATH_CAPITAIS_FULL             = "/Users/gabrielribeirobizerril/Documents/GitHub/llm/editai_extractor_llm_based/resultados/output_capitais_pt2.xlsx"
ABA_CAPITAIS_SIMPLIFICADO           = "dados consolidados - nao mudar"
DATA_PATH_CAPITAIS_SIMPLIFICADO     = "/Users/gabrielribeirobizerril/Documents/GitHub/llm/editai_extractor_llm_based/resultados/output_capitais_simplificado.xlsx"
DATA_PATH_ESTADOS                   = "/Users/gabrielribeirobizerril/Documents/GitHub/llm/editai_extractor_llm_based/resultados/estados_output.xlsx"
DATA_PATH_PAINEL_PNAB               = "/Users/gabrielribeirobizerril/Documents/GitHub/llm/editai_extractor_llm_based/resultados/Adesão - Politica Nacional Aldir Blanc.xlsx"
CAPITAL_ESTADO                      = "/Users/gabrielribeirobizerril/Documents/GitHub/llm/editai_extractor_llm_based/tratamento/estado_capital.json"
NAME_NAN                            = ["não previu", "indisponível", "none", "ok", "nan", "na"]

### Utils

In [ ]:
# Funções auxiliares
def load_data(file_path: str, sheet_name: str ):
    return pd.read_excel(io=file_path, sheet_name=sheet_name)


def normalizar(texto):
    if pd.isna(texto):  # trata valores NaN
        return texto
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join(c for c in texto if not unicodedata.combining(c))  # remove acentos
    return texto.upper().strip()


def tratar_colunas_numericas(df, colunas, name_nan):
    for coluna in colunas:
        df[coluna] = df[coluna].apply(
            lambda x: np.nan if str(x).strip().lower() in name_nan else x
        )
    return df

def normalizar_nome_coluna(nome):
    # Se já está todo minúsculo e só tem letras, números e _
    if re.fullmatch(r"[a-z0-9_]+", nome):
        return nome
    
    # Caso contrário, normaliza
    nome = nome.strip().lower()
    nome = re.sub(r"\s+", "_", nome)  # espaços -> _
    nome = re.sub(r"[^\w_]", "", nome)  # remove acentos e caracteres especiais
    
    # Remover acentos
    import unicodedata
    nome = ''.join(
        c for c in unicodedata.normalize('NFKD', nome)
        if not unicodedata.combining(c)
    )
    
    return nome

def get_instrumento(df: pd.DataFrame, coluna: str):
    df["instrumento_fomento"] = df[coluna].apply(
        lambda x: x.split("_")[2].split(".")[0]
    )
    return df

def calcula_valor_vaga_por_cota(df: pd.DataFrame):
    for categoria in ["negras", "indigenas", "pcd"]:
        df[f"valor_{categoria}"] = df[f"cotas_{categoria}"] * df["valor_total"]
        df[f"vagas_{categoria}"] = df[f"cotas_{categoria}"] * df["vagas_totais"]
    
    for coluna in df.columns:
        if "vagas" in coluna:
            df[coluna] = df[coluna].round(0)
            df[coluna] = df[coluna].astype("Int64")
    
    return df

def get_cubo(df: pd.DataFrame):
    agrupado = df.groupby(["ente", "ente_uf"]).agg(
        qtd_pdfs                = ("pdf", "nunique"),
        valor_total_edital      = ("valor_total", "sum"),
        vagas_totais_edital     = ("vagas_totais", "sum"),
        c_vagas_negras          = ("vagas_negras", "sum"),
        c_valor_negras          = ("valor_negras", "sum"),
        c_vagas_indigenas       = ("vagas_indigenas", "sum"),
        c_valor_indigenas       = ("valor_indigenas", "sum"),
        c_vagas_pcd             = ("vagas_pcd", "sum"),
        c_valor_pcd             = ("valor_pcd", "sum"),
        i_vagas_fomento         = ("fomento_vagas", "sum"),
        i_valor_fomento         = ("fomento_valor", "sum"),
        i_vagas_premio          = ("premio_vagas", "sum"),
        i_valor_premio          = ("premio_valor", "sum"),
        i_vagas_bolsa           = ("bolsa_vagas", "sum"),
        i_valor_bolsa           = ("bolsa_valor", "sum"),
        i_vagas_subsidio        = ("subsidio_vagas", "sum"),
        i_valor_subsidio        = ("subsidio_valor", "sum"),
        i_vagas_cultura_viva    = ("culturaviva_vagas", "sum"),
        i_valor_cultura_viva    = ("culturaviva_valor", "sum"),
        qtd_culturaviva_pdf     = ("culturaviva_pdf", "sum"),
        qtd_fomento_pdf         = ("fomento_pdf", "sum"),
        qtd_premio_pdf          = ("premio_pdf", "sum"),
        qtd_subsidio_pdf        = ("subsidio_pdf", "sum"),
        qtd_bolsa_pdf           = ("bolsa_pdf ", "sum")
    ).reset_index()

    return agrupado

def arredonda_valores(df: pd.DataFrame):
    # Seleciona colunas que contêm "valor" no nome
    colunas_valor = [col for col in df.columns if "valor" in col.lower()]

    # Arredonda todas para 2 casas decimais
    df[colunas_valor] = df[colunas_valor].round(2)

    return df


In [81]:
# Carrega tabela da PNAB
df_pnab = load_data(file_path=DATA_PATH_PAINEL_PNAB, sheet_name="Adesao a Política")
df_pnab = df_pnab.iloc[3:].reset_index(drop=True)
# Define a primeira linha restante como novo cabeçalho
df_pnab.columns = df_pnab.iloc[0]
# Remove a linha agora redundante (que virou cabeçalho)
df_pnab = df_pnab.iloc[1:].reset_index(drop=True)
df_pnab = df_pnab.drop(index=0).reset_index(drop=True)
# Filtra ciclo 1 
df_pnab = df_pnab.loc[df_pnab["Ano de adesão"] == "Ciclo 1"]
# Filtra apenas Município
df_pnab = df_pnab.loc[df_pnab["Tipo de Ente"] == "Município"]
# Normaliza entes
df_pnab["ente"]     = df_pnab["Ente Federativo"].apply(normalizar)
df_pnab["uf"]       = df_pnab["UF"].apply(normalizar)
df_pnab["ente_uf"]  = df_pnab["ente"] + "-" + df_pnab["uf"]
df_pnab["ente_uf"]  = df_pnab["ente_uf"].str.strip().str.upper()
# Define o escopo dos dados de plano de ação dos entes
plano_de_acao = df_pnab[["ente_uf","Valor do Plano de Ação"]]

In [82]:
# Carrega Capitais Completa
df_capitais_full            = load_data(file_path=DATA_PATH_CAPITAIS_FULL, sheet_name=ABA_CAPITAIS_FULL)
df_capitais_full            = df_capitais_full[:-1]
df_capitais_full = (
    df_capitais_full
    .sort_values(
        by=["uf", "valor_total", "vagas_totais"],
        ascending=[True, True, True] 
    )
    .reset_index(drop=True)
)
# Carrega Capitais Simplificada
df_capitais_simplificado    = load_data(file_path=DATA_PATH_CAPITAIS_SIMPLIFICADO, sheet_name=ABA_CAPITAIS_SIMPLIFICADO)
df_capitais_simplificado = (
    df_capitais_simplificado
    .sort_values(
        by=["uf", "valor_total", "vagas_totais"],
        ascending=[True, True, True]  
    )
    .reset_index(drop=True)
)


In [83]:
# Cria tabela consolidada com os caminhos dos pdf
df_capitais_consolidado = df_capitais_simplificado.merge(
    df_capitais_full[["pdf"]],
    left_index=True,
    right_index=True,
    how="left"
)

In [84]:
df_capitais_consolidado.loc[89, "pdf"] = "2024-05_PORTOALEGRE_FOMENTO.pdf"

In [85]:
# Seleciona colunas que contêm "valor" no nome
colunas_valor = [col for col in df_capitais_consolidado.columns if "valor" in col.lower()]

# Arredonda todas para 2 casas decimais
df_capitais_consolidado[colunas_valor] = df_capitais_consolidado[colunas_valor].round(2)

In [86]:
# Cria a coluna "instrumento_fomento"
df_capitais_consolidado = get_instrumento(df=df_capitais_consolidado, coluna="pdf")

In [87]:
# Carrega Estados e UF e Cria ID de relacionamento com base PNAB
with open(CAPITAL_ESTADO, "r", encoding="utf-8") as f:
    capitais_para_uf = json.load(f)

# Normaliza colunas de relacnionamento
df_capitais_consolidado                         = df_capitais_consolidado.rename(columns={"uf":"ente"})
df_capitais_consolidado["ente"]                 = df_capitais_consolidado["ente"].str.strip().str.upper()
df_capitais_consolidado["ente"]                 = df_capitais_consolidado["ente"].apply(normalizar)
df_capitais_consolidado["uf"]                   = df_capitais_consolidado["ente"].map(capitais_para_uf)
df_capitais_consolidado["ente_uf"]              = df_capitais_consolidado["ente"] + "-" + df_capitais_consolidado["uf"]
df_capitais_consolidado["ente_uf"]              = df_capitais_consolidado["ente_uf"].str.strip().str.upper()
df_capitais_consolidado["instrumento_fomento"]  = df_capitais_consolidado["instrumento_fomento"].apply(normalizar)


In [88]:
# Tratamento para instrumento de fomento 
df_pivot_n = (
    df_capitais_consolidado.pivot_table(
        index=df_capitais_consolidado.index,  
        columns="instrumento_fomento",
        values=["pdf"],
        aggfunc="nunique"
    )
)

# Ajusta o nome das colunas para o formato desejado
df_pivot_n.columns = [
    f"{col2.lower()}_{col1.split('_')[0]}"
    for col1, col2 in df_pivot_n.columns
]

# # Junta a tabela pivot com a tabela base
df_capitais_consolidado = df_capitais_consolidado.join(df_pivot_n)

In [89]:
df_capitais_consolidado.columns

Index(['ente', 'valor_total', 'cotas_negras', 'cotas_indigenas', 'cotas_pcd',
       'vagas_totais', 'Edital específico', 'TAG PN', 'TAG PI', 'TAG PCD',
       'pdf', 'instrumento_fomento', 'uf', 'ente_uf', 'bolsa_pdf',
       'culturaviva_pdf', 'fomento_pdf', 'premio_pdf', 'subsidio_pdf'],
      dtype='object')

In [90]:
# Reordena as colunas
df_capitais_consolidado = df_capitais_consolidado[[
    'ente',
    'uf',
    'ente_uf',
    'pdf',
    'instrumento_fomento',
    'valor_total', 
    'vagas_totais', 
    'cotas_negras', 
    'cotas_indigenas', 
    'cotas_pcd',
    'Edital específico', 
    'TAG PN', 
    'TAG PI', 
    'TAG PCD',
    'bolsa_pdf',
    'culturaviva_pdf', 
    'fomento_pdf', 
    'premio_pdf', 
    'subsidio_pdf'
]]

In [91]:
# Normaliza o nome das colunas
df_capitais_consolidado.columns = [normalizar_nome_coluna(c) for c in df_capitais_consolidado.columns]

In [92]:
df_capitais_consolidado.columns = [normalizar_nome_coluna(c) for c in df_capitais_consolidado.columns]

In [93]:
df_capitais_consolidado.columns

Index(['ente', 'uf', 'ente_uf', 'pdf', 'instrumento_fomento', 'valor_total',
       'vagas_totais', 'cotas_negras', 'cotas_indigenas', 'cotas_pcd',
       'edital_especifico', 'tag_pn', 'tag_pi', 'tag_pcd', 'bolsa_pdf',
       'culturaviva_pdf', 'fomento_pdf', 'premio_pdf', 'subsidio_pdf'],
      dtype='object')

In [94]:
# Tratamento das colunas com valores faltantes
colunas_cotas = [
    "cotas_negras", 
    "cotas_indigenas", 
    "cotas_pcd"
]
colunas_vagas_valores = [
    "vagas_totais", 
    "valor_total"
]

df_capitais_consolidado = tratar_colunas_numericas(
    df_capitais_consolidado, 
    colunas_cotas+colunas_vagas_valores, 
    NAME_NAN
)



In [95]:
# Tratamento das colunas numéricas
for coluna in colunas_cotas:
    df_capitais_consolidado[coluna] = df_capitais_consolidado[coluna].astype(float)
    df_capitais_consolidado[coluna] = df_capitais_consolidado[coluna]/100

df_capitais_consolidado["vagas_totais"] = (
    df_capitais_consolidado["vagas_totais"].astype("Int64")
)



In [96]:
df_capitais_consolidado["bolsa_pdf"].sum()

np.float64(5.0)

In [97]:
# Calculo de valor e vagas por cotas
df_capitais_consolidado = calcula_valor_vaga_por_cota(df=df_capitais_consolidado)
    

In [98]:
# Faz arredondamento novamente
df_capitais_consolidado = arredonda_valores(df=df_capitais_consolidado)

In [99]:
cotas_valor = (df_capitais_consolidado["valor_pcd"].sum()) + (df_capitais_consolidado["valor_negras"].sum()) + (df_capitais_consolidado["valor_indigenas"].sum())
valort_total = df_capitais_consolidado["valor_total"].sum()
cotas_valor - valort_total

np.float64(-147550125.53999996)

In [100]:
# Tratamento para instrumento de fomento 
df_pivot = (
    df_capitais_consolidado.pivot_table(
        index=df_capitais_consolidado.index,  
        columns="instrumento_fomento",
        values=["vagas_totais", "valor_total"],
        aggfunc="sum"
    )
)

# Ajusta o nome das colunas para o formato desejado
df_pivot.columns = [
    f"{col2.lower()}_{col1.split('_')[0]}"  # Ex: 'BOLSA' -> 'bolsa_vagas'
    if col1 == "vagas_totais" else f"{col2.lower()}_valor"
    for col1, col2 in df_pivot.columns
]

# Junta a tabela pivot com a tabela base
df_capitais_consolidado_final = df_capitais_consolidado.join(df_pivot)

In [101]:
df_capitais_consolidado_final

,ente,uf,ente_uf,pdf,instrumento_fomento,valor_total,vagas_totais,cotas_negras,cotas_indigenas,cotas_pcd,...,bolsa_vagas,culturaviva_vagas,fomento_vagas,premio_vagas,subsidio_vagas,bolsa_valor,culturaviva_valor,fomento_valor,premio_valor,subsidio_valor
0,ARACAJU,SE,ARACAJU-SE,2024-09_ARACAJU_BOLSA.pdf.pdf,BOLSA,250000.00,5,NaN,NaN,NaN,...,5,<NA>,<NA>,<NA>,<NA>,250000.0,NaN,NaN,NaN,NaN
1,ARACAJU,SE,ARACAJU-SE,2024-12_ARACAJU_CULTURAVIVA.pdf.pdf,CULTURAVIVA,330085.32,12,0.25,0.1,0.05,...,<NA>,12,<NA>,<NA>,<NA>,NaN,330085.32,NaN,NaN,NaN
2,ARACAJU,SE,ARACAJU-SE,2024-11_ARACAJU_CULTURAVIVA.pdf.pdf,CULTURAVIVA,800000.00,10,0.25,0.1,0.05,...,<NA>,10,<NA>,<NA>,<NA>,NaN,800000.00,NaN,NaN,NaN
3,ARACAJU,SE,ARACAJU-SE,2025-06_ARACAJU_FOMENTO.pdf,FOMENTO,2420000.00,60,0.25,0.1,0.05,...,<NA>,<NA>,60,<NA>,<NA>,NaN,NaN,2420000.0,NaN,NaN
4,BELEM,PA,BELEM-PA,2024-04_BELÉM_FOMENTO.pdf.pdf,FOMENTO,5224089.00,100,0.25,0.1,0.05,...,<NA>,<NA>,100,<NA>,<NA>,NaN,NaN,5224089.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,VITORIA,ES,VITORIA-ES,2024-05_VITÓRIA_SUBSÍDIO.pdf,SUBSIDIO,210000.00,18,0.25,0.1,0.05,...,<NA>,<NA>,<NA>,<NA>,18,NaN,NaN,NaN,NaN,210000.0
139,VITORIA,ES,VITORIA-ES,2024-03_VITÓRIA_FOMENTO.pdf,FOMENTO,300000.00,4,NaN,NaN,NaN,...,<NA>,<NA>,4,<NA>,<NA>,NaN,NaN,300000.0,NaN,NaN
140,VITORIA,ES,VITORIA-ES,2024-07_VITÓRIA_FOMENTO.pdf,FOMENTO,420000.00,7,0.25,0.1,NaN,...,<NA>,<NA>,7,<NA>,<NA>,NaN,NaN,420000.0,NaN,NaN
141,VITORIA,ES,VITORIA-ES,2024-04_VITÓRIA_FOMENTO.pdf,FOMENTO,600000.00,50,0.25,0.1,0.05,...,<NA>,<NA>,50,<NA>,<NA>,NaN,NaN,600000.0,NaN,NaN


In [76]:
# Define o cubo com os valores agregados
df_cubo = get_cubo(df_capitais_consolidado_final)
df_cubo = df_cubo.sort_values(by="ente", ascending=True)

KeyError: "Column(s) ['bolsa_pdf '] do not exist"

In [77]:
# Enriquece o cubo com valores do repasse
df_merged = pd.merge(df_cubo, plano_de_acao, how='left', on='ente_uf')
df_merged = df_merged.rename(columns={
    "Valor do Plano de Ação": "valor_repasse"
})

In [25]:
# Seleciona colunas que contêm "valor" no nome
colunas_valor = [col for col in df_merged.columns if "valor" in col.lower()]

# Arredonda todas para 2 casas decimais
df_merged[colunas_valor] = df_merged[colunas_valor].round(2)

In [26]:
# Exportação
df_merged.to_excel("./output/cubo_capitais_8_9_15_13.xlsx")
df_capitais_consolidado_final.to_excel("./output/tabela_capitais_consolidada_8_9_15_1.xlsx")

In [27]:
with pd.ExcelWriter("./output/tabela_capitais_completa_8_9_17_26.xlsx") as writer:
    df_capitais_consolidado_final.to_excel(writer, sheet_name="tabela", index=False)
    df_merged.to_excel(writer, sheet_name="cubo", index=False)

In [ ]:
df_merged

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
len(df_capitais_consolidado_final["ente"].unique())

26

In [ ]:
df_capitais_consolidado_final["valor_total"].sum() == df_capitais_consolidado_final["valor_total"].sum()

np.True_

In [ ]:
df_capitais_consolidado_final["vagas_totais"].sum() == df_capitais_consolidado_final["vagas_totais"].sum()

np.True_

In [ ]:
df_capitais_consolidado_final

,ente,uf,ente_uf,pdf,instrumento_fomento,valor_total,vagas_totais,cotas_negras,cotas_indigenas,cotas_pcd,...,bolsa_vagas,culturaviva_vagas,fomento_vagas,premio_vagas,subsidio_vagas,bolsa_valor,culturaviva_valor,fomento_valor,premio_valor,subsidio_valor
0,ARACAJU,SE,ARACAJU-SE,2024-09_ARACAJU_BOLSA.pdf.pdf,BOLSA,250000.00,5,NaN,NaN,NaN,...,5,<NA>,<NA>,<NA>,<NA>,250000.0,NaN,NaN,NaN,NaN
1,ARACAJU,SE,ARACAJU-SE,2024-12_ARACAJU_CULTURAVIVA.pdf.pdf,CULTURAVIVA,330085.32,12,0.25,0.1,0.05,...,<NA>,12,<NA>,<NA>,<NA>,NaN,330085.32,NaN,NaN,NaN
2,ARACAJU,SE,ARACAJU-SE,2024-11_ARACAJU_CULTURAVIVA.pdf.pdf,CULTURAVIVA,800000.00,10,0.25,0.1,0.05,...,<NA>,10,<NA>,<NA>,<NA>,NaN,800000.00,NaN,NaN,NaN
3,ARACAJU,SE,ARACAJU-SE,2025-06_ARACAJU_FOMENTO.pdf,FOMENTO,2420000.00,60,0.25,0.1,0.05,...,<NA>,<NA>,60,<NA>,<NA>,NaN,NaN,2420000.0,NaN,NaN
4,BELEM,PA,BELEM-PA,2024-04_BELÉM_FOMENTO.pdf.pdf,FOMENTO,5224089.00,100,0.25,0.1,0.05,...,<NA>,<NA>,100,<NA>,<NA>,NaN,NaN,5224089.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,VITORIA,ES,VITORIA-ES,2024-05_VITÓRIA_SUBSÍDIO.pdf,SUBSIDIO,210000.00,18,0.25,0.1,0.05,...,<NA>,<NA>,<NA>,<NA>,18,NaN,NaN,NaN,NaN,210000.0
139,VITORIA,ES,VITORIA-ES,2024-03_VITÓRIA_FOMENTO.pdf,FOMENTO,300000.00,4,NaN,NaN,NaN,...,<NA>,<NA>,4,<NA>,<NA>,NaN,NaN,300000.0,NaN,NaN
140,VITORIA,ES,VITORIA-ES,2024-07_VITÓRIA_FOMENTO.pdf,FOMENTO,420000.00,7,0.25,0.1,NaN,...,<NA>,<NA>,7,<NA>,<NA>,NaN,NaN,420000.0,NaN,NaN
141,VITORIA,ES,VITORIA-ES,2024-04_VITÓRIA_FOMENTO.pdf,FOMENTO,600000.00,50,0.25,0.1,0.05,...,<NA>,<NA>,50,<NA>,<NA>,NaN,NaN,600000.0,NaN,NaN


In [30]:
instrumento = (df_capitais_consolidado_final["bolsa_pdf"].sum())+(df_capitais_consolidado_final["culturaviva_pdf"].sum())+(df_capitais_consolidado_final["premio_pdf"].sum())+(df_capitais_consolidado_final["subsidio_pdf"].sum())+(df_capitais_consolidado_final["fomento_pdf"].sum())

In [31]:
instrumento

np.float64(143.0)

In [32]:
instrumento_agrupado = 37+73+17+11

In [33]:
instrumento_agrupado

138

In [ ]:
instrumento==df_merged["qtd_pdfs"].sum()

np.True_

In [ ]:
agrupado = df_capitais_consolidado_final.groupby(["ente", "ente_uf", "instrumento_fomento"]).agg(
        qtd_pdfs                = ("pdf", "nunique"),
        valor_total_edital      = ("valor_total", "sum"),
        vagas_totais_edital     = ("vagas_totais", "sum"),
        c_vagas_negras          = ("vagas_negras", "sum"),
        c_valor_negras          = ("valor_negras", "sum"),
        c_vagas_indigenas       = ("vagas_indigenas", "sum"),
        c_valor_indigenas       = ("valor_indigenas", "sum"),
        c_vagas_pcd             = ("vagas_pcd", "sum"),
        c_valor_pcd             = ("valor_pcd", "sum"),
        i_vagas_fomento         = ("fomento_vagas", "sum"),
        i_valor_fomento         = ("fomento_valor", "sum"),
        i_vagas_premio          = ("premio_vagas", "sum"),
        i_valor_premio          = ("premio_valor", "sum"),
        i_vagas_bolsa           = ("bolsa_vagas", "sum"),
        i_valor_bolsa           = ("bolsa_valor", "sum"),
        i_vagas_subsidio        = ("subsidio_vagas", "sum"),
        i_valor_subsidio        = ("subsidio_valor", "sum"),
        i_vagas_cultura_viva    = ("culturaviva_vagas", "sum"),
        i_valor_cultura_viva    = ("culturaviva_valor", "sum")
    ).reset_index()


In [ ]:
agrupado

,ente,ente_uf,instrumento_fomento,qtd_pdfs,valor_total_edital,vagas_totais_edital,c_vagas_negras,c_valor_negras,c_vagas_indigenas,c_valor_indigenas,...,i_vagas_fomento,i_valor_fomento,i_vagas_premio,i_valor_premio,i_vagas_bolsa,i_valor_bolsa,i_vagas_subsidio,i_valor_subsidio,i_vagas_cultura_viva,i_valor_cultura_viva
0,ARACAJU,ARACAJU-SE,BOLSA,1,250000.00,5,0,0.00,0,0.00,...,0,0.0,0,0.00,5,250000.0,0,0.0,0,0.00
1,ARACAJU,ARACAJU-SE,CULTURAVIVA,2,1130085.32,22,5,282521.33,2,113008.53,...,0,0.0,0,0.00,0,0.0,0,0.0,22,1130085.32
2,ARACAJU,ARACAJU-SE,FOMENTO,1,2420000.00,60,15,605000.00,6,242000.00,...,60,2420000.0,0,0.00,0,0.0,0,0.0,0,0.00
3,BELEM,BELEM-PA,FOMENTO,1,5224089.00,100,25,1306022.25,10,522408.90,...,100,5224089.0,0,0.00,0,0.0,0,0.0,0,0.00
4,BELO HORIZONTE,BELO HORIZONTE-MG,CULTURAVIVA,1,4060000.00,124,31,1015000.00,12,406000.00,...,0,0.0,0,0.00,0,0.0,0,0.0,124,4060000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,TERESINA,TERESINA-PI,FOMENTO,1,3500000.00,37,9,875000.00,4,350000.00,...,37,3500000.0,0,0.00,0,0.0,0,0.0,0,0.00
72,TERESINA,TERESINA-PI,PREMIO,1,353008.73,71,18,88252.18,7,35300.87,...,0,0.0,71,353008.73,0,0.0,0,0.0,0,0.00
73,VITORIA,VITORIA-ES,CULTURAVIVA,1,610558.70,21,5,152639.67,2,61055.87,...,0,0.0,0,0.00,0,0.0,0,0.0,21,610558.70
74,VITORIA,VITORIA-ES,FOMENTO,4,1520000.00,67,16,305000.00,7,122000.00,...,67,1520000.0,0,0.00,0,0.0,0,0.0,0,0.00


In [ ]:
# Tratamento para instrumento de fomento 
df_pivot_n = (
    df_capitais_consolidado.pivot_table(
        index=df_capitais_consolidado.index,  
        columns="instrumento_fomento",
        values=["vagas_totais", "valor_total"],
        aggfunc="sum"
    )
)

# Ajusta o nome das colunas para o formato desejado
df_pivot_n.columns = [
    f"{col2.lower()}_{col1.split('_')[0]}"  # Ex: 'BOLSA' -> 'bolsa_vagas'
    if col1 == "vagas_totais" else f"{col2.lower()}_valor"
    for col1, col2 in df_pivot_n.columns
]

# Junta a tabela pivot com a tabela base
df_capitais_consolidado_final = df_capitais_consolidado.join(df_pivot_n)